# Step 1: fine-tune LLM using top result from (fixed) ranker

In [ ]:
def launch():
    import os
    import StepOne
    import torch

    os.environ['MODEL_TYPE'] = 'xxl'
    augment = int(os.environ.get('AUGMENT', '1'))
    os.environ['BATCH_SIZE'] = '1'
    
    world_size = torch.cuda.device_count()
    torch.multiprocessing.spawn(StepOne.step_one,
                                args=(world_size,),
                                nprocs=world_size,
                                join=True)
    
launch()

# Step 2: learn ranker using (fixed pre-finetuned) task LLM

In [ ]:
def launch():
    import os
    import StepTwo
    import torch

    os.environ['MODEL_TYPE'] = 'base'
    augment = int(os.environ.get('AUGMENT', '1'))
    os.environ['BATCH_SIZE'] = '32'
    os.environ['LEARN_BATCH_SIZE'] = '32'
    os.environ['STEP1_ITER'] = '2_augment2'
    
    world_size = torch.cuda.device_count()
    torch.multiprocessing.spawn(StepTwo.step_two,
                                args=(world_size,),
                                nprocs=world_size,
                                join=True)
    
launch()

# Step 3: Prepare Submission Files

In [ ]:
def test_stepone(*, step1_iter, k):
    import json
    from TaskLLM import TaskLLM
    from PersonalizedProductRating import dev_loader, test_loader
    from ProgressPrinter import ProgressPrinter
    from transformers import T5ForConditionalGeneration
    import torch
    from Util import interleave
    
    device = 'cuda'
    torch.set_default_device(device)
    torch.manual_seed(2112)

    dev = dev_loader(batch_size=8)
    test = test_loader(batch_size=8)

    t5 = T5ForConditionalGeneration.from_pretrained('google/flan-t5-base')
    t5.load_adapter(f'User_keq{k}_t5base_step1_iter{step1_iter}', 'taskllm')
    t5.enable_adapters()
    
    taskllm = TaskLLM(t5=t5, adapter_name="taskllm")

    def inner_batch(func, inner_batch_size, inputs):
        from more_itertools import chunked
        return [ func(*ib) for ib in zip(*[ chunked(g, inner_batch_size) for g in inputs ]) ]

    print(f'*** step1_iter: {step1_iter} ***')
    
    with ProgressPrinter(f'{k} MAE (dev)', f'{k} MSE (dev)') as printer:
        cumsum = lambda z, acc=0: [0] + [ acc := acc + v for v in z ]

        devgolds, testgolds = [], []
        
        for isdev, (examples, labels) in interleave(dev, test, sequential=True):
            with torch.no_grad():
                texts_to_embed = [ [ text[:256]
                                     for text in (' '.join(ex['review'].split()), )
                                   ] +
                                   [ text[:256]
                                     for v in ex['profile']
                                     for text in (' '.join(v['text'].split()), )
                                   ]
                                   for ex in examples
                                 ]
                embeddings = torch.cat(inner_batch(func = dev.embed,
                                                   inner_batch_size = 128,
                                                   inputs = (sum(texts_to_embed, []),)
                                                  ),
                                       dim=0)
                splits = cumsum(map(len, texts_to_embed))
                indices = [ torch.topk(embeddings[a,:] @ embeddings[a+1:b,:].T, dim=0, k=k).indices for a, b in zip(splits, splits[1:]) ]
                prompts = [ dev.prepend_to_prompt(ex, [ ex['profile'][ind] for ind in index.to('cpu').tolist() ])
                            for ex, index in zip(examples, indices) ]
                cumul = taskllm.predict(prompts).exp().cumsum(dim=1)
                guesses = (cumul>=0.5).long().argmax(dim=1)

                if isdev:
                    targets = [ int(label) - 1 for label in labels ]
                    targets = torch.Tensor(targets).long().to(guesses.device)
                    mae = torch.abs(guesses - targets).float().mean().item()
                    mse = torch.square(guesses - targets).float().mean().item()

                for ex, guess in zip(examples, guesses):
                    (devgolds if isdev else testgolds).append({ 'id': ex['id'], 'output': f'{1+guess}' })

            printer.addobs(mae, mse)

        printer.print()
        printer.autoprint = False

        for wut, golds in ( ('dev', devgolds), ('test', testgolds) ):
            with open(f'lamp3u_{wut}golds_t5base_keq{k}_step1_iter{step1_iter}.json', 'w') as jsonfile:
                json.dump({ 'task': 'LaMP_3', 'golds': golds }, jsonfile)

test_stepone(step1_iter='2_augment2', k=4)

In [ ]:
def prepare_submission_probensemble(*, nvoters, step2_iter, step1_iter, k):
    import json
    from RewardPredictor import RewardPredictor
    from TaskLLM import TaskLLM
    from PersonalizedProductRating import dev_loader, test_loader
    from ProgressPrinter import ProgressPrinter
    from transformers import T5ForConditionalGeneration
    import torch
    from Util import interleave
    
    device = 'cuda'
    torch.set_default_device(device)
    torch.manual_seed(8675309)

    dev = dev_loader(batch_size=8)
    test = test_loader(batch_size=8)

    t5 = T5ForConditionalGeneration.from_pretrained('google/flan-t5-base')
    t5.load_adapter(f'User_keq{k}_t5base_step1_iter{step1_iter}', 'taskllm')
    t5.load_adapter(f'User_keq{k}_t5base_step2_iter{step2_iter}', 'rhat')
    t5.enable_adapters()
    
    taskllm = TaskLLM(t5=t5, adapter_name="taskllm")
    rewardpredictor = RewardPredictor(t5=t5, adapter_name="rhat", model_id=f'User_keq{k}_t5base_step2_iter{step2_iter}')

    gumbel = torch.distributions.gumbel.Gumbel(0,1)
    def randomized_similarity(embeddings, nsamples):
        scores = embeddings[0,:] @ embeddings[1:,:].T
        temperature = scores[0].item() - scores[min(scores.shape[0], 4)].item()
        gumbel_shape = torch.Size([nsamples, scores.shape[0]])
        gumbels = temperature * gumbel.sample(gumbel_shape).to(scores.device)
        return torch.unique(torch.topk(scores.unsqueeze(0) + gumbels, dim=1, k=k).indices, sorted=False, dim=0)

    def inner_batch(func, inner_batch_size, inputs):
        from more_itertools import chunked
        return [ func(*ib) for ib in zip(*[ chunked(g, inner_batch_size) for g in inputs ]) ]

    print(f'*** step1_iter: {step1_iter} step2_iter: {step2_iter} nvoters {nvoters} ***')
    
    with ProgressPrinter(f'{k} MAE (dev)', f'{k} MSE (dev)') as printer:
        devgolds, testgolds = [], []
        cumsum = lambda z, acc=0: [0] + [ acc := acc + v for v in z ]

        for isdev, (examples, labels) in interleave(dev, test, sequential=True):
            with torch.no_grad():
                texts_to_embed = [ [ text[:256]
                                     for text in (' '.join(ex['review'].split()), )
                                   ] +
                                   [ text[:256]
                                     for v in ex['profile']
                                     for text in (' '.join(v['text'].split()), )
                                   ]
                                   for ex in examples
                                 ]
                embeddings = torch.cat(inner_batch(func = dev.embed,
                                                   inner_batch_size = 128,
                                                   inputs = (sum(texts_to_embed, []),)
                                                  ),
                                       dim=0)
                splits = cumsum(map(len, texts_to_embed))
                randos = [ randomized_similarity(embeddings[a:b,:], 64) for a, b in zip(splits, splits[1:]) ]
                prompts = [ [ dev.prepend_to_prompt(ex, [ ex['profile'][ind] for ind in indices ])
                              for indices in rando.to('cpu').tolist()
                            ]
                            for ex, rando in zip(examples, randos)
                          ]
                rhats = torch.cat(inner_batch(func = rewardpredictor.predict,
                                              inner_batch_size = 128,
                                              inputs = (sum(prompts, []),)
                                             ),
                                  dim=0)
                splits = cumsum(map(len, prompts))
                votingprompts = [ [ prompt[v] for v in torch.topk(rhats[a:b].view(-1), k=min(nvoters, b-a)).indices.to('cpu').tolist() ]
                                    for a, b, prompt in zip(splits, splits[1:], prompts)
                                ]
                predicts = torch.cat(inner_batch(func = taskllm.predict,
                                                 inner_batch_size = 128,
                                                 inputs = (sum(votingprompts, []),)
                                                ),
                                     dim=0)
                splits = cumsum(map(len, votingprompts))
                guesses = torch.cat([ (predicts[a:b,:].logsumexp(dim=0, keepdim=True).exp().cumsum(dim=1) >= 0.5).long().argmax(dim=1)
                                      for a, b in zip(splits, splits[1:])
                                    ],
                                    dim=0)

                if isdev:
                    targets = [ int(label) - 1 for label in labels ]
                    targets = torch.Tensor(targets).long().to(guesses.device)
                    mae = torch.abs(guesses - targets).float().mean().item()
                    mse = torch.square(guesses - targets).float().mean().item()
                else:
                    mae, mse = None, None

                for ex, guess in zip(examples, guesses):
                    (devgolds if isdev else testgolds).append({ 'id': ex['id'], 'output': f'{1+guess}' })

            printer.addobs(mae, mse)

        printer.print()
        printer.autoprint = False

        for wut, golds in ( ('dev', devgolds), ('test', testgolds) ):
            with open(f'lamp3u_{wut}golds_t5base_keq{k}_step1_iter{step1_iter}_step2_iter{step2_iter}_nvoters{nvoters}.json', 'w') as jsonfile:
                json.dump({ 'task': 'LaMP_3', 'golds': golds }, jsonfile)
            
for nvoters in [1, 3, 5]:
    prepare_submission_probensemble(k=4, step1_iter='2_augment2', step2_iter='0_augment2', nvoters=nvoters)